In [13]:
from pyspark.sql import SparkSession, functions as F, Window

spark = SparkSession.builder \
    .appName("Footbal project") \
    .getOrCreate()

results_df = spark.read.option("header", "true").csv("results.csv")
goals_df = spark.read.option("header", "true").csv("goalscorers.csv")

results_df.show(5)
goals_df.show(5)


+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|      date|home_team|away_team|home_score|away_score|tournament|   city| country|neutral|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|1872-11-30| Scotland|  England|         0|         0|  Friendly|Glasgow|Scotland|  FALSE|
|1873-03-08|  England| Scotland|         4|         2|  Friendly| London| England|  FALSE|
|1874-03-07| Scotland|  England|         2|         1|  Friendly|Glasgow|Scotland|  FALSE|
|1875-03-06|  England| Scotland|         2|         2|  Friendly| London| England|  FALSE|
|1876-03-04| Scotland|  England|         3|         0|  Friendly|Glasgow|Scotland|  FALSE|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
only showing top 5 rows
+----------+---------+---------+---------+----------------+------+--------+-------+
|      date|home_team|away_team|     team|          scorer|minute|own_goa

Prvi zadatak: najbolja i najgora gol razlika Srbije 

In [ ]:
rdd = results_df.rdd.map(lambda row: (
    row['home_team'], 
    row['away_team'], 
    int(row['home_score']), 
    int(row['away_score'])
))

serbia_rdd = rdd.filter(lambda x: x[0] == 'Serbia' or x[1] == 'Serbia')

def map_goal_diff(match):
    home, away, home_score, away_score = match
    if home == 'Serbia':
        opponent = away
        diff = home_score - away_score
    else:
        opponent = home
        diff = away_score - home_score
    return (opponent, diff)

mapped = serbia_rdd.map(map_goal_diff)
reduced = mapped.reduceByKey(lambda a, b: a + b)

best = reduced.max(key=lambda x: x[1])
worst = reduced.min(key=lambda x: x[1])

print("Najbolja gol razlika sa Srbijom:", best)
print("Najgora gol razlika sa Srbijom:", worst)

Najbolja gol razlika:
Najgora gol razlika:

Drugi zadatak: Fudbaleri sa het-trikom na Evropskom

In [ ]:
df2 = goals_df.join(results_df, on=["date", "home_team", "away_team"], how="left")

euro_df = df2.filter(F.col("tournament") == "UEFA Euro")

hat_tricks = (
    euro_df.groupBy("date", "home_team", "away_team", "team", "scorer", "tournament", "country")
    .agg(F.count("*").alias("golova"))
    .filter(F.col("golova") >= 3)
)

hat_tricks.select(
    F.col("date").alias("Datum"),
    F.col("home_team").alias("Domaći"),
    F.col("away_team").alias("Gosti"),
    F.col("team").alias("Igra za"),
    F.col("scorer").alias("Strelac"),
    F.col("golova").alias("Golova na utakmici"),
    F.col("tournament").alias("Turnir"),
    F.col("country").alias("Zemlja")
).show(truncate=False)

+----------+-----------+-----------+-----------+----------------+------------------+---------+-----------+
|Datum     |Domaći     |Gosti      |Igra za    |Strelac         |Golova na utakmici|Turnir   |Zemlja     |
+----------+-----------+-----------+-----------+----------------+------------------+---------+-----------+
|1984-06-19|France     |Yugoslavia |France     |Michel Platini  |3                 |UEFA Euro|France     |
|1984-06-16|France     |Belgium    |France     |Michel Platini  |3                 |UEFA Euro|France     |
|2000-06-25|Netherlands|Serbia     |Netherlands|Patrick Kluivert|3                 |UEFA Euro|Netherlands|
|1976-06-17|Yugoslavia |Germany    |Germany    |Dieter Müller   |3                 |UEFA Euro|Yugoslavia |
|2000-06-20|Portugal   |Germany    |Portugal   |Sérgio Conceição|3                 |UEFA Euro|Netherlands|
|1988-06-15|England    |Netherlands|Netherlands|Marco van Basten|3                 |UEFA Euro|Germany    |
|1980-06-14|Germany    |Netherlands|G

+----------+-----------+-----------+-----------+----------------+------------------+---------+-----------+
|Datum     |Domaći     |Gosti      |Igra za    |Strelac         |Golova na utakmici|Turnir   |Zemlja     |
+----------+-----------+-----------+-----------+----------------+------------------+---------+-----------+
|1984-06-19|France     |Yugoslavia |France     |Michel Platini  |3                 |UEFA Euro|France     |
|1984-06-16|France     |Belgium    |France     |Michel Platini  |3                 |UEFA Euro|France     |
|2000-06-25|Netherlands|Serbia     |Netherlands|Patrick Kluivert|3                 |UEFA Euro|Netherlands|
|1976-06-17|Yugoslavia |Germany    |Germany    |Dieter Müller   |3                 |UEFA Euro|Yugoslavia |
|2000-06-20|Portugal   |Germany    |Portugal   |Sérgio Conceição|3                 |UEFA Euro|Netherlands|
|1988-06-15|England    |Netherlands|Netherlands|Marco van Basten|3                 |UEFA Euro|Germany    |
|1980-06-14|Germany    |Netherlands|Germany    |Klaus Allofs    |3                 |UEFA Euro|Italy      |
|2008-06-10|Spain      |Russia     |Spain      |David Villa     |3                 |UEFA Euro|Austria    |
+----------+-----------+-----------+-----------+----------------+------------------+---------+-----------+


Treci zadatak: Najbolji strelci na svetskom od 1990.

In [31]:
df3 = goals_df.join(results_df, on=["date", "home_team", "away_team"], how="left")

df_wc = df3.filter(F.col("tournament") == "FIFA World Cup") \
        .withColumn("godina", F.year(F.to_date(F.col("date"), "yyyy-MM-dd"))) \
        .filter(F.col("godina") >= 1990)

scorers = df_wc.groupBy("scorer", "team", "godina") \
        .agg(F.count("*").alias("Golova"),
             F.first("tournament").alias("Turnir"),
             F.first("country").alias("Zemlja"))

window = Window.partitionBy("godina").orderBy(F.desc("Golova"))

top_scorers = scorers.withColumn("rang", F.row_number().over(window)) \
                     .filter(F.col("rang") <= 2) \
                     .orderBy("godina", F.desc("Golova"))


top_scorers.select(
    F.col("scorer").alias("Strelac"),
    F.col("team").alias("Reprezentacija"),
    F.col("Golova"),
    F.col("Rang"),
    F.col("Godina"),
    F.col("Turnir"),
    F.col("Zemlja")
).show(truncate=False)

+-------------------+--------------+------+----+------+--------------+-------------+
|Strelac            |Reprezentacija|Golova|Rang|Godina|Turnir        |Zemlja       |
+-------------------+--------------+------+----+------+--------------+-------------+
|Salvatore Schillaci|Italy         |6     |1   |1990  |FIFA World Cup|Italy        |
|Tomáš Skuhravý     |Czechoslovakia|5     |2   |1990  |FIFA World Cup|Italy        |
|Hristo Stoichkov   |Bulgaria      |6     |1   |1994  |FIFA World Cup|United States|
|Oleg Salenko       |Russia        |6     |2   |1994  |FIFA World Cup|United States|
|Davor Šuker        |Croatia       |6     |1   |1998  |FIFA World Cup|France       |
|Christian Vieri    |Italy         |5     |2   |1998  |FIFA World Cup|France       |
|Ronaldo            |Brazil        |8     |1   |2002  |FIFA World Cup|South Korea  |
|Miroslav Klose     |Germany       |5     |2   |2002  |FIFA World Cup|Japan        |
|Miroslav Klose     |Germany       |5     |1   |2006  |FIFA World

+-------------------+--------------+------+----+------+--------------+-------------+
|Strelac            |Reprezentacija|Golova|Rang|Godina|Turnir        |Zemlja       |
+-------------------+--------------+------+----+------+--------------+-------------+
|Salvatore Schillaci|Italy         |6     |1   |1990  |FIFA World Cup|Italy        |
|Tomáš Skuhravý     |Czechoslovakia|5     |2   |1990  |FIFA World Cup|Italy        |
|Hristo Stoichkov   |Bulgaria      |6     |1   |1994  |FIFA World Cup|United States|
|Oleg Salenko       |Russia        |6     |2   |1994  |FIFA World Cup|United States|
|Davor Šuker        |Croatia       |6     |1   |1998  |FIFA World Cup|France       |
|Christian Vieri    |Italy         |5     |2   |1998  |FIFA World Cup|France       |
|Ronaldo            |Brazil        |8     |1   |2002  |FIFA World Cup|South Korea  |
|Miroslav Klose     |Germany       |5     |2   |2002  |FIFA World Cup|Japan        |
|Miroslav Klose     |Germany       |5     |1   |2006  |FIFA World Cup|Germany      |
|David Villa        |Spain         |3     |2   |2006  |FIFA World Cup|Germany      |
|David Villa        |Spain         |5     |1   |2010  |FIFA World Cup|South Africa |
|Diego Forlán       |Uruguay       |5     |2   |2010  |FIFA World Cup|South Africa |
|James Rodríguez    |Colombia      |6     |1   |2014  |FIFA World Cup|Brazil       |
|Thomas Müller      |Germany       |5     |2   |2014  |FIFA World Cup|Brazil       |
|Harry Kane         |England       |6     |1   |2018  |FIFA World Cup|Russia       |
|Antoine Griezmann  |France        |4     |2   |2018  |FIFA World Cup|Russia       |
|Kylian Mbappé      |France        |8     |1   |2022  |FIFA World Cup|Qatar        |
|Lionel Messi       |Argentina     |7     |2   |2022  |FIFA World Cup|Qatar        |
+-------------------+--------------+------+----+------+--------------+-------------+
